This is a tutorial for the DFODE-kit package.

In [85]:
import os

import numpy as np
import matplotlib.pyplot as plt

from dfode_kit.df_interface import (
    OneDFreelyPropagatingFlameConfig,
    setup_one_d_flame_case,
    df_to_h5,
)
from dfode_kit.data_operations import (
    touch_h5, 
    get_TPY_from_h5, 
    random_perturb,
    label_npy,
    integrate_h5,
)
from dfode_kit.dfode_core.model.mlp import MLP
from dfode_kit.utils import BCT

DFODE_ROOT = os.environ['DFODE_ROOT']

### A brief introduction to the DFODE method

#### Low-dimensional manifold sampling

A key challenge in preparing training data is achieving sufficient coverage of the relevant thermochemical composition space, which is often prohibitively high-dimensional when detailed chemistry involves tens to hundreds of species. 

To address this, DFODE-kit adopts a low-dimensional
manifold sampling strategy, where thermochemical states are extracted from canonical flame configurations that retain the essential topology of high-dimensional turbulent flames. This approach ensures both computational efficiency and physical representativeness of the training datasets.

In this tutorial, we will demonstrate how to use DFODE-kit to sample a low-dimensional manifold of thermochemical states from a one-dimensional laminar freely propagating flame simulated with DeepFlame. The following code block could also be found in `case_init.ipynb` files within the case templates provides in the `cases` directory. It is used to initialize the simulation and update the dictionary files for the simulation.

In [86]:
# Operating condition settings
config_dict = {
    "mechanism": f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
    "T0": 300,
    "p0": 101325,
    "fuel": "H2:1",
    "oxidizer": "O2:0.21,N2:0.79",
    "eq_ratio": 1.0,
}
config = OneDFreelyPropagatingFlameConfig(**config_dict)

# Simulation settings
settings = {
    "sim_time_step": 1e-6,
    "sim_write_interval": 1e-5,
    "num_output_steps": 10,
}
config.update_config(settings)

# Setup the case and update dictionary files
setup_one_d_flame_case(config, '.')

Solving premixed flame...
Laminar Flame Speed      :   2.3489328863 m/s
Laminar Flame Thickness  :   0.0003694362 m
One-dimensional flame case setup completed at: /data1/kexiao/projects/dfode_project/DFODE-kit/tutorials/oneD_freely_propagating_flame


Note that at the point, the simulation is not yet started. The user would need to ensure a working version of DeepFlame is available and run the `Allrun` script from command line to start the simulation.

```bash
./Allrun
```

After the simulation is completed, we proceed to use DFODE-kit to gather and manage the thermochemical data.

In [87]:
df_to_h5(
    root_dir=f"{DFODE_ROOT}/tutorials/oneD_freely_propagating_flame",
    mechanism=f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
    hdf5_file_path=f"{DFODE_ROOT}/tutorials/oneD_freely_propagating_flame/tutorial_data.h5",
    include_mesh=True,
)

# The above is equivalent to the following cli command:
# dfode-kit sample --mech Burke2012_s9r23.yaml \
#     --case tutorials/oneD_freely_propagating_flame \
#     --save tutorials/oneD_freely_propagating_flame/tutorial_data.h5 \
#     --include-mesh

Species names: ['T', 'p', 'H', 'H2', 'O', 'OH', 'H2O', 'O2', 'HO2', 'H2O2', 'N2']
Saved concatenated arrays to /data1/kexiao/projects/dfode_project/DFODE-kit/tutorials/oneD_freely_propagating_flame/tutorial_data.h5


Checking the contents of the h5 file

In [88]:
touch_h5("tutorial_data.h5")

Inspecting HDF5 file: tutorial_data.h5

Metadata in the HDF5 file:
mechanism: /data1/kexiao/projects/dfode_project/DFODE-kit/mechanisms/Burke2012_s9r23.yaml
root_directory: /data1/kexiao/projects/dfode_project/DFODE-kit/tutorials/oneD_freely_propagating_flame
species_names: ['T' 'p' 'H' 'H2' 'O' 'OH' 'H2O' 'O2' 'HO2' 'H2O2' 'N2']

Groups and datasets in the HDF5 file:
Group: mesh
  Dataset: Cx, Shape: (500, 1)
  Dataset: Cy, Shape: (500, 1)
  Dataset: Cz, Shape: (500, 1)
  Dataset: V, Shape: (500, 1)
Group: scalar_fields
  Dataset: 0.0001, Shape: (500, 11)
  Dataset: 0.00011, Shape: (500, 11)
  Dataset: 1e-05, Shape: (500, 11)
  Dataset: 2e-05, Shape: (500, 11)
  Dataset: 3e-05, Shape: (500, 11)
  Dataset: 4e-05, Shape: (500, 11)
  Dataset: 5e-05, Shape: (500, 11)
  Dataset: 6e-05, Shape: (500, 11)
  Dataset: 7e-05, Shape: (500, 11)
  Dataset: 8e-05, Shape: (500, 11)
  Dataset: 9e-05, Shape: (500, 11)


#### Data augmentation and labeling

While laminar canonical flames provide fundamental thermochemical states,their trajectory-aligned sampling in composition space poses significant limitations for a posteriori modeling applications. First, these sampled states are confined to predefined flamelet manifolds, making the trained model highly sensitive to perturbations and leading to an over-constrained representation. Second, the sampled states span a lower-dimensional subspace, which fails to encompass the full range of thermochemical variations encountered in turbulent combustion. As a result, the model becomes vulnerable to off-manifold perturbations—deviations from the training manifold that frequently arise in turbulent reacting flows.

To tackle this challenge, a data augmentation strategy is employed, where collected states are perturbed to simulate the effects of multi-dimensional transport and turbulence disturbances.

In [89]:
thermochemical_data = get_TPY_from_h5("tutorial_data.h5")
print(thermochemical_data[0])

aug_thermochemical_data = random_perturb(thermochemical_data)
print(aug_thermochemical_data[0])

np.save("tutorial_data_aug.npy", aug_thermochemical_data)

# The above is equivalent to the following cli command:
# dfode-kit augment --h5_file tutorial_data.h5 \
#     --output_file tutorial_data_aug.npy

Number of datasets in scalar_fields group: 11
[3.00023e+02 1.02883e+05 1.50501e-41 2.85116e-02 1.63881e-47 9.68647e-48
 8.28493e-48 2.26269e-01 5.70836e-38 2.00401e-52 7.45219e-01]
[4.34898431e+02 1.03023870e+05 1.30235941e-44 3.73487006e-02
 3.76889810e-47 3.09947306e-51 2.86370158e-49 2.17432299e-01
 1.17072095e-35 5.19600513e-56 7.45219000e-01]


The CVODE integrator from Cantera is used for time integration and to provide supervised learning labels.

In [90]:
labeled_data = label_npy(
    mech_path=f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
    time_step=1e-6,
    source_path="tutorial_data_aug.npy",
)

print(f'{labeled_data.shape=}')
print(labeled_data[0])
np.save("tutorial_data_labeled.npy", labeled_data)

# The above is equivalent to the following cli command:
# dfode-kit label --mech Burke2012_s9r23.yaml \
#     --time 1e-6 \
#     --source tutorial_data_aug.npy \
#     --save tutorial_data_labeled.npy

Loaded dataset from: tutorial_data_aug.npy
test_data.shape=(5500, 11)
Total time used: 0.93 seconds
labeled_data.shape=(5500, 22)
[4.34898431e+02 1.03023870e+05 1.30235941e-44 3.73487006e-02
 3.76889810e-47 3.09947306e-51 2.86370158e-49 2.17432299e-01
 1.17072095e-35 5.19600513e-56 7.45219000e-01 4.34898431e+02
 1.03023870e+05 6.04320390e-29 3.73487006e-02 1.27860203e-32
 6.60575312e-33 7.74753499e-33 2.17432299e-01 9.72749461e-27
 1.52164666e-36 7.45219000e-01]


#### Model training

Only a demo for training a model is provided here.

In [91]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model instantiation
demo_model = MLP([thermochemical_data.shape[1], 400, 400, 400, 400, thermochemical_data.shape[1]-3]).to(device)

# Data loading
thermochem_states1 = labeled_data[:, 0:11]
thermochem_states2 = labeled_data[:, 11:]
thermochem_states1[:, 2:] = np.clip(thermochem_states1[:, 2:], 0, 1)
thermochem_states2[:, 2:] = np.clip(thermochem_states2[:, 2:], 0, 1)

features = torch.tensor(BCT(thermochem_states1), dtype=torch.float32).to(device)
labels = torch.tensor(BCT(thermochem_states2[:, 2:-1]) - BCT(thermochem_states1[:, 2:-1]), dtype=torch.float32).to(device)

features_mean = torch.mean(features, dim=0)
features_std = torch.std(features, dim=0)
features = (features - features_mean) / features_std

labels_mean = torch.mean(labels, dim=0)
labels_std = torch.std(labels, dim=0)
labels = (labels - labels_mean) / labels_std

# Training
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(demo_model.parameters(), lr=1e-3)

demo_model.train()  
for epoch in range(100):
    optimizer.zero_grad()
    preds = demo_model(features)
    loss = criterion(preds, labels)
    loss.backward()
    optimizer.step()
    
    print("Epoch: {}, Loss: {}".format(epoch, loss.item()))

torch.save(
    {
        'net': demo_model.state_dict(),
        'data_in_mean': features_mean.cpu().numpy(),
        'data_in_std': features_std.cpu().numpy(),
        'data_target_mean': labels_mean.cpu().numpy(),
        'data_target_std': labels_std.cpu().numpy(),
    },
    "demo_model.pt"
)

Epoch: 0, Loss: 0.5456070303916931
Epoch: 1, Loss: 0.5291376113891602
Epoch: 2, Loss: 0.5356059074401855
Epoch: 3, Loss: 0.5246405601501465
Epoch: 4, Loss: 0.5228965878486633
Epoch: 5, Loss: 0.5245605111122131
Epoch: 6, Loss: 0.5222760438919067
Epoch: 7, Loss: 0.5164884924888611
Epoch: 8, Loss: 0.5139819383621216
Epoch: 9, Loss: 0.5166049003601074
Epoch: 10, Loss: 0.5146443247795105
Epoch: 11, Loss: 0.5114165544509888
Epoch: 12, Loss: 0.5117149949073792
Epoch: 13, Loss: 0.5111161470413208
Epoch: 14, Loss: 0.5108492970466614
Epoch: 15, Loss: 0.5098523497581482
Epoch: 16, Loss: 0.5081254243850708
Epoch: 17, Loss: 0.5057773590087891
Epoch: 18, Loss: 0.5046067833900452
Epoch: 19, Loss: 0.5039052367210388
Epoch: 20, Loss: 0.5029950737953186
Epoch: 21, Loss: 0.501594603061676
Epoch: 22, Loss: 0.5002539753913879
Epoch: 23, Loss: 0.4999106824398041
Epoch: 24, Loss: 0.49878478050231934
Epoch: 25, Loss: 0.49734410643577576
Epoch: 26, Loss: 0.4964168071746826
Epoch: 27, Loss: 0.49534714221954346


#### Model testing

Model testing is closely associated with a specific testing dataset, enabling the evaluation of the trained models’ performance. DFODE-kit can directly operate on HDF5 files, adhering to a predefined format that facilitates seamless data integration.

In [92]:
model_settings = {
    'model_path': "demo_model.pt",
    'device': 'cpu',
    'model_class': MLP,
    'model_layers': [thermochemical_data.shape[1], 400, 400, 400, 400, thermochemical_data.shape[1]-3],
    'time_step': 1e-6,
    'mech': f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml"
}
integrate_h5("tutorial_data.h5", 1e-6, nn_integration=False, model_settings=model_settings)

touch_h5("tutorial_data.h5")

[1.00000000e-06 3.00023000e+02 1.02883000e+05 1.93180336e-41
 2.85116114e-02 1.79077845e-47 1.07332764e-47 8.89073823e-48
 2.26269091e-01 6.21366281e-38 2.13981345e-52 7.45219298e-01]
[1.00000000e-06 2.39472000e+03 1.02881031e+05 7.49624466e-05
 1.22141488e-03 3.92780145e-04 5.72190086e-03 2.40164611e-01
 7.20252466e-03 1.74150538e-06 1.89174168e-07 7.45219875e-01]
[1.00000000e-06 3.00027000e+02 1.02907000e+05 1.93286729e-41
 2.85116114e-02 1.78235919e-47 1.06644844e-47 8.84346360e-48
 2.26269091e-01 6.19316244e-38 2.15120479e-52 7.45219298e-01]
[1.00000000e-06 2.39477000e+03 1.02903030e+05 7.49969233e-05
 1.22168596e-03 3.92980850e-04 5.72353273e-03 2.40161469e-01
 7.20395248e-03 1.74209961e-06 1.89277964e-07 7.45219451e-01]
[1.00000000e-06 3.00001000e+02 1.01330000e+05 1.91643344e-41
 2.85116114e-02 9.46442269e-48 7.60276420e-48 2.76612710e-48
 2.26269091e-01 3.32684024e-38 4.66902696e-53 7.45219298e-01]
[1.00000000e-06 2.38895000e+03 1.01699005e+05 7.50156151e-05
 1.22005459e-03 3.9